# Join DataBases

12 July, 2024

@roman

In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import norm
import h3
from tqdm import tqdm
from scipy.spatial import KDTree


In [2]:
# Settings
# show 100 columns in pandas
pd.set_option('display.max_columns', 500)

---
# Load Data

## Properties

In [3]:
# vars to stay
cols_to_stay_properties = [
    'property_id', 'valor_fisico_construccion', 'fecha_avaluo', 'colonia',
    'id_clase_inmueble', 'id_tipo_inmueble', 'conservacion', 'elevador', 'niveles',
    'cve_ref_proximidad_urbana', 'cve_nivel_equipamiento_urbano', 'valor_mercado',
    'valor_concluido_total', 'nivel', 'unidades_rentables', 'edad_anios', 'vida_util_remanente',
    'cve_clasificacion_zona', 'cve_vigilancia', 'regimen_propiedad', 'tipo_vialidad',
    'id_municipio', 'id_entidad_f', 'cp', 'recamaras', 'banos', 'medio_banos', 'estacionamiento',
    'superficie_terreno', 'superficie_construida', 'superficie_accesoria', 'superficie_vendible',
    'valor_fisico_terreno_m2', 'property_type', 'city_cluster', 'distance_to_ocean',
    'ocean_label', 'longitud', 'latitud'
]

# read parquet
gdf_properties = pd.read_parquet("../../data/interim/cleaned_data_s7.parquet")
gdf_properties = gdf_properties[cols_to_stay_properties]

# to geopandas
gdf_properties = gpd.GeoDataFrame(
    gdf_properties,
    geometry=gpd.points_from_xy(gdf_properties['longitud'], gdf_properties['latitud']),
    crs="EPSG:4326"
    )

# change crs to 6372
gdf_properties = gdf_properties.to_crs("EPSG:6372")

# see
print(gdf_properties.shape)
gdf_properties.head()

(852913, 40)


,property_id,valor_fisico_construccion,fecha_avaluo,colonia,id_clase_inmueble,id_tipo_inmueble,conservacion,elevador,niveles,cve_ref_proximidad_urbana,cve_nivel_equipamiento_urbano,valor_mercado,valor_concluido_total,nivel,unidades_rentables,edad_anios,vida_util_remanente,cve_clasificacion_zona,cve_vigilancia,regimen_propiedad,tipo_vialidad,id_municipio,id_entidad_f,cp,recamaras,banos,medio_banos,estacionamiento,superficie_terreno,superficie_construida,superficie_accesoria,superficie_vendible,valor_fisico_terreno_m2,property_type,city_cluster,distance_to_ocean,ocean_label,longitud,latitud,geometry
0,66cf10199ef943a5a5ff82129e53d5d7,1611371.52,2020-05-27,SD,4,2,4.0,1.0,4,1,1,4305275,4305000,3,1,25,45,4,NaN,PRIVADA INDIVIDUAL,NaN,003,09,4600,2,1,0,0,143,348,0,348,8918.0,house,0,inf,no_ocean,-99.158103,19.310875,POINT (2797822.489 815572.062)
1,f84f9a0c784f491eab6bb100b513a95b,460588.28,2020-07-23,SD,3,2,4.0,2.0,3,2,1,1351682,1352000,3,1,25,35,3,NaN,PRIVADA INDIVIDUAL,NaN,058,15,57100,2,2,0,1,113,108,0,108,6151.0,house,0,inf,no_ocean,-99.065361,19.491722,POINT (2807131.800 835725.131)
2,7ddd5a94feed4955a809f986d95722d8,1029107.14,2020-10-05,SD,4,2,4.0,2.0,1,2,1,2544900,2545000,0,1,1,69,4,NaN,PRIVADA INDIVIDUAL,NaN,121,15,54743,3,2,0,1,200,170,0,170,5620.0,house,0,inf,no_ocean,-99.213541,19.633393,POINT (2791321.912 851047.443)
3,d2e75fb3b385461995bf8d34b9d1fdfb,598308.77,2020-10-22,SD,4,2,4.0,2.0,2,2,1,2906000,2906000,3,1,35,35,4,NaN,PRIVADA INDIVIDUAL,NaN,003,09,4480,6,2,0,0,107,139,0,139,18149.0,house,0,inf,no_ocean,-99.114525,19.333061,POINT (2802339.433 818111.830)
4,6f1194f9853443219e63f4a3222b010e,144720.00,2020-05-27,SD,4,2,4.0,2.0,1,3,1,535697,536000,3,1,26,44,4,NaN,PRIVADA INDIVIDUAL,NaN,058,15,57100,2,1,0,0,84,54,0,54,4544.8,house,0,inf,no_ocean,-99.052833,19.486586,POINT (2808454.380 835185.302)


## ENIGH

### Concentrado Hogar

In [4]:
# cols 2 stay
columns_to_stay_hogar_enigh = [
    "ubica_geo", "year",
    "tam_loc", "est_socio", 
    "clase_hog", "sexo_jefe", "edad_jefe", "educa_jefe", "tot_integ", "mayores",
    "menores", "p12_64", "p65mas", "percep_ing", "ing_cor", "estim_alqu", 
    "otros_ing", "gasto_mon", "vivienda", "alquiler", "pred_cons", "cuidados",
]

# read enigh
df_concentrado_hogar_enigh = pd.read_csv("../../data/interim/concentrado_hogar_enigh.csv")
df_concentrado_hogar_enigh = df_concentrado_hogar_enigh[columns_to_stay_hogar_enigh]

# see
print(df_concentrado_hogar_enigh.shape)
df_concentrado_hogar_enigh.head()


(1023, 22)


,ubica_geo,year,tam_loc,est_socio,clase_hog,sexo_jefe,edad_jefe,educa_jefe,tot_integ,mayores,menores,p12_64,p65mas,percep_ing,ing_cor,estim_alqu,otros_ing,gasto_mon,vivienda,alquiler,pred_cons,cuidados
0,1001,2018,1.313242,2.830593,2.0,1.0,50.089514,6.479373,3.721097,2.999458,0.721640,2.691494,0.307964,2.391824,63956.301534,6351.823899,46.323383,39119.615453,3114.482487,1146.301370,175.330648,1727.998294
1,1001,2020,1.191967,2.848673,2.0,1.0,50.307018,6.518359,3.511532,2.877479,0.634053,2.578268,0.299212,2.295080,60912.502414,6937.108693,78.762806,37361.636895,3523.484832,1395.231161,184.045724,1552.910254
2,1001,2022,1.260126,2.843422,2.0,1.0,50.853007,6.667584,3.485117,2.844748,0.640369,2.536844,0.307904,2.319125,85273.975500,8849.450081,115.145540,47870.292160,4335.689380,1732.631168,335.986748,2252.075444
3,1005,2018,3.006706,2.289274,2.0,1.0,45.461916,6.186617,4.025102,3.090068,0.935034,2.915252,0.174815,2.538047,62425.257471,4832.711173,13.453221,40447.472278,2864.566049,902.798797,209.100271,1647.925020
4,1005,2020,2.915232,2.345722,2.0,1.0,47.396102,5.892137,3.988493,3.215378,0.773114,3.048939,0.166440,2.517314,58610.447087,5442.595426,136.200860,35552.268076,3085.935918,1084.445514,199.979916,1682.611930


### Viviendas

In [5]:
# important cols
columns_to_stay_vivienda_enigh = [
    "ubica_geo", "year",
    "tipo_viv", "antiguedad", "cuart_dorm", "tenencia", "renta",
    "estim_pago", "pago_viv", "pago_mesp", "tipo_adqui", "viv_usada", "tipo_finan",
    "escrituras", "tot_resid", "tot_hom", "tot_muj", "tot_hog", "total_viviendas"
]

# read viviendas
df_viviendas_enigh = pd.read_csv("../../data/interim/viviendas_enigh.csv")
df_viviendas_enigh = df_viviendas_enigh[columns_to_stay_vivienda_enigh]

# see
print(df_viviendas_enigh.shape)
df_viviendas_enigh.head()

(1023, 19)


,ubica_geo,year,tipo_viv,antiguedad,cuart_dorm,tenencia,renta,estim_pago,pago_viv,pago_mesp,tipo_adqui,viv_usada,tipo_finan,escrituras,tot_resid,tot_hom,tot_muj,tot_hog,total_viviendas
0,1001,2018,1.0,19.145343,2.460400,4.0,385.384175,2165.050435,442.032669,0.362238,1.0,0.166781,1.0,1.0,3.781875,1.806226,1.975650,1.019717,234164.0
1,1001,2020,1.0,18.578944,2.355871,4.0,484.117574,2361.336658,670.729401,0.385572,1.0,0.161022,1.0,1.0,3.555936,1.693341,1.862595,1.016633,267473.0
2,1001,2022,1.0,20.415628,2.393433,4.0,594.349359,3013.785845,711.650391,0.386612,1.0,0.179857,1.0,1.0,3.514820,1.645770,1.869050,1.010339,267713.0
3,1005,2018,1.0,17.957064,2.421640,4.0,316.383800,1650.605073,640.096362,0.365966,1.0,0.103256,5.0,1.0,4.099135,2.150613,1.948522,1.024653,26691.0
4,1005,2020,1.0,16.014364,2.221359,4.0,360.672807,1863.259378,362.239594,0.303057,1.0,0.123321,5.0,1.0,4.024456,1.997624,2.026832,1.010092,37455.0


### Merge

In [6]:
# enigh data
df_enigh = pd.merge(
    df_concentrado_hogar_enigh,
    df_viviendas_enigh,
    on=["ubica_geo", "year"],
    how="inner"
)

# zfill 5 to ubica_geo
df_enigh["ubica_geo"] = df_enigh["ubica_geo"].astype(str).str.zfill(5)

# see
print(df_enigh.shape)
df_enigh.head()

(1023, 39)


,ubica_geo,year,tam_loc,est_socio,clase_hog,sexo_jefe,edad_jefe,educa_jefe,tot_integ,mayores,menores,p12_64,p65mas,percep_ing,ing_cor,estim_alqu,otros_ing,gasto_mon,vivienda,alquiler,pred_cons,cuidados,tipo_viv,antiguedad,cuart_dorm,tenencia,renta,estim_pago,pago_viv,pago_mesp,tipo_adqui,viv_usada,tipo_finan,escrituras,tot_resid,tot_hom,tot_muj,tot_hog,total_viviendas
0,01001,2018,1.313242,2.830593,2.0,1.0,50.089514,6.479373,3.721097,2.999458,0.721640,2.691494,0.307964,2.391824,63956.301534,6351.823899,46.323383,39119.615453,3114.482487,1146.301370,175.330648,1727.998294,1.0,19.145343,2.460400,4.0,385.384175,2165.050435,442.032669,0.362238,1.0,0.166781,1.0,1.0,3.781875,1.806226,1.975650,1.019717,234164.0
1,01001,2020,1.191967,2.848673,2.0,1.0,50.307018,6.518359,3.511532,2.877479,0.634053,2.578268,0.299212,2.295080,60912.502414,6937.108693,78.762806,37361.636895,3523.484832,1395.231161,184.045724,1552.910254,1.0,18.578944,2.355871,4.0,484.117574,2361.336658,670.729401,0.385572,1.0,0.161022,1.0,1.0,3.555936,1.693341,1.862595,1.016633,267473.0
2,01001,2022,1.260126,2.843422,2.0,1.0,50.853007,6.667584,3.485117,2.844748,0.640369,2.536844,0.307904,2.319125,85273.975500,8849.450081,115.145540,47870.292160,4335.689380,1732.631168,335.986748,2252.075444,1.0,20.415628,2.393433,4.0,594.349359,3013.785845,711.650391,0.386612,1.0,0.179857,1.0,1.0,3.514820,1.645770,1.869050,1.010339,267713.0
3,01005,2018,3.006706,2.289274,2.0,1.0,45.461916,6.186617,4.025102,3.090068,0.935034,2.915252,0.174815,2.538047,62425.257471,4832.711173,13.453221,40447.472278,2864.566049,902.798797,209.100271,1647.925020,1.0,17.957064,2.421640,4.0,316.383800,1650.605073,640.096362,0.365966,1.0,0.103256,5.0,1.0,4.099135,2.150613,1.948522,1.024653,26691.0
4,01005,2020,2.915232,2.345722,2.0,1.0,47.396102,5.892137,3.988493,3.215378,0.773114,3.048939,0.166440,2.517314,58610.447087,5442.595426,136.200860,35552.268076,3085.935918,1084.445514,199.979916,1682.611930,1.0,16.014364,2.221359,4.0,360.672807,1863.259378,362.239594,0.303057,1.0,0.123321,5.0,1.0,4.024456,1.997624,2.026832,1.010092,37455.0


In [7]:
# see lengths of ubica_geo
df_enigh["ubica_geo"].str.len().value_counts()

ubica_geo
5    1023
Name: count, dtype: int64

^^^
note: see if all properties have a match in enigh

## Banxico

### INPC

In [8]:
# date
min_year = gdf_properties["fecha_avaluo"].min()

# important cols
columns_to_stay_inpc = [
    "date", "inpc", "em_avg_pct_change_inpc"
]

# read indexes
df_inpc = pd.read_csv("../../data/interim/inpc.csv")
df_inpc = df_inpc[columns_to_stay_inpc]

# see
print(df_inpc.shape)
df_inpc.head()

(101, 3)


,date,inpc,em_avg_pct_change_inpc
0,2016-01-01,89.386381,NaN
1,2016-02-01,89.777781,0.004379
2,2016-03-01,89.910001,0.003548
3,2016-04-01,89.625278,0.001630
4,2016-05-01,89.225615,-0.000110


In [9]:
# wrangle
df_inpc = (
    df_inpc
    .assign(
        lag_year_inpc=lambda x: x['inpc'].shift(12),
        lag_3months_em_avg_pct_change_inpc=lambda x: x['em_avg_pct_change_inpc'].shift(3),
        date=lambda x: pd.to_datetime(x['date'])
    )
    .query("date >= @min_year")
    .reset_index(drop=True)
    .drop(columns=["inpc", "em_avg_pct_change_inpc"])
)

# see
print(df_inpc.shape)
df_inpc.head()

(65, 3)


,date,lag_year_inpc,lag_3months_em_avg_pct_change_inpc
0,2019-01-01,98.795000,0.004269
1,2019-02-01,99.171374,0.005480
2,2019-03-01,99.492157,0.005917
3,2019-04-01,99.154847,0.004470
4,2019-05-01,98.994080,0.003113


### Minimum Wage

In [10]:
# important cols
columns_to_stay_wage = [
    "date", "general_minimum_wage", "em_avg_general"
]

# read indexes
df_wage = pd.read_csv("../../data/interim/minimum_wage.csv")
df_wage = df_wage[columns_to_stay_wage]

# see
print(df_wage.shape)
df_wage.head()

(89, 3)


,date,general_minimum_wage,em_avg_general
0,2017-01-01,99.6963,NaN
1,2017-02-01,99.1235,NaN
2,2017-03-01,98.5190,NaN
3,2017-04-01,98.3980,NaN
4,2017-05-01,98.5158,NaN


In [11]:
# wrangle
df_wage = (
    df_wage
    .assign(
        lag_year_general_minimum_wage=lambda x: x['general_minimum_wage'].shift(12),
        lag_3months_em_avg_general=lambda x: x['em_avg_general'].shift(3),
        date=lambda x: pd.to_datetime(x['date'])
    )
    .query("date >= @min_year")
    .reset_index(drop=True)
    .drop(columns=["general_minimum_wage", "em_avg_general"])
)

# see
print(df_wage.shape)
df_wage.head()

(65, 3)


,date,lag_year_general_minimum_wage,lag_3months_em_avg_general
0,2019-01-01,104.2765,0.052312
1,2019-02-01,103.8808,0.052859
2,2019-03-01,103.5459,0.024591
3,2019-04-01,103.8981,0.049981
4,2019-05-01,104.0668,0.069418


### Merge

In [12]:
# merge
df_banxico = (
    pd.merge(
        df_inpc,
        df_wage,
        on="date",
        how="inner"
    ).rename(columns={
            "lag_3months_em_avg_general": "lag_3months_em_avg_general_wage",
    })
)

# see
print(df_banxico.shape)
df_banxico.head()

(65, 5)


,date,lag_year_inpc,lag_3months_em_avg_pct_change_inpc,lag_year_general_minimum_wage,lag_3months_em_avg_general_wage
0,2019-01-01,98.795000,0.004269,104.2765,0.052312
1,2019-02-01,99.171374,0.005480,103.8808,0.052859
2,2019-03-01,99.492157,0.005917,103.5459,0.024591
3,2019-04-01,99.154847,0.004470,103.8981,0.049981
4,2019-05-01,98.994080,0.003113,104.0668,0.069418


In [13]:
# see dates
print(df_banxico["date"])

0    2019-01-01
1    2019-02-01
2    2019-03-01
3    2019-04-01
4    2019-05-01
        ...    
60   2024-01-01
61   2024-02-01
62   2024-03-01
63   2024-04-01
64   2024-05-01
Name: date, Length: 65, dtype: datetime64[ns]


## Competitors

In [14]:
# important cols
columns_to_stay_competitors = [
    "property_id", "competitors_weighted_mean_log_price_per_sqm", "competitors_weighted_std_log_price_per_sqm",
    "num_competitors"
]

# read parquet
df_competitors_stats = pd.read_parquet("../../data/misc/df_competitors_stats.parquet")
df_competitors_stats = df_competitors_stats[columns_to_stay_competitors]

# add confidence interval at 95%
z975 = norm.ppf(0.975)
df_competitors_stats = (
    df_competitors_stats
    .assign(
        competitors_weighted_mean_log_price_per_sqm_lower=lambda x: x["competitors_weighted_mean_log_price_per_sqm"] - z975 * x["competitors_weighted_std_log_price_per_sqm"] / np.sqrt(x["num_competitors"]),
        competitors_weighted_mean_log_price_per_sqm_upper=lambda x: x["competitors_weighted_mean_log_price_per_sqm"] + z975 * x["competitors_weighted_std_log_price_per_sqm"] / np.sqrt(x["num_competitors"])
    )
    .assign(
        # if nan then fill with mean for the case the std is 0 or null
        competitors_weighted_mean_log_price_per_sqm_lower=lambda x: np.where(
            x["competitors_weighted_mean_log_price_per_sqm_lower"].isna(),
            x["competitors_weighted_mean_log_price_per_sqm"],
            x["competitors_weighted_mean_log_price_per_sqm_lower"]
        ),
        competitors_weighted_mean_log_price_per_sqm_upper=lambda x: np.where(
            x["competitors_weighted_mean_log_price_per_sqm_upper"].isna(),
            x["competitors_weighted_mean_log_price_per_sqm"],
            x["competitors_weighted_mean_log_price_per_sqm_upper"]
        )
    )
    .drop(columns=["competitors_weighted_std_log_price_per_sqm", "num_competitors"])
)


# see
print(df_competitors_stats.shape)
df_competitors_stats.head()

(730843, 4)


,property_id,competitors_weighted_mean_log_price_per_sqm,competitors_weighted_mean_log_price_per_sqm_lower,competitors_weighted_mean_log_price_per_sqm_upper
0,0002b8fe92544e4b8af409bbb9f4a86b,9.381862,9.327027,9.436697
1,00056d55e77146738ff4850e9559e1ce,9.861810,9.745271,9.978350
2,0018f6096c7b4595868cf3eb5e6b0341,9.802367,9.567079,10.037655
3,001a550115e540309f4bfabf1291f701,9.450411,9.401777,9.499045
4,001aa6fcab2244978a9e993833dcfb8b,10.035896,9.633672,10.438120


## DENUE

In [15]:
# important columns
columns_to_stay_denue = [
    'Fecha_Alta', 'category', 'geometry'
]

# get denue data
gdf_denue = gpd.read_parquet("../../data/misc/denue_data.parquet")
gdf_denue = gdf_denue[columns_to_stay_denue]

# geometry to crs
gdf_denue = gdf_denue.to_crs("EPSG:6372")

# see
print(gdf_denue.shape)
gdf_denue.head()

(14225, 3)


,Fecha_Alta,category,geometry
0,2010-07-01,supermarkets,POINT (2566629.977 964916.350)
1,2010-07-01,supermarkets,POINT (2791472.433 841445.827)
2,2010-07-01,supermarkets,POINT (2785414.263 825108.458)
3,2010-07-01,supermarkets,POINT (2795636.961 813163.543)
4,2010-07-01,supermarkets,POINT (2494203.156 822602.670)


In [16]:
# counts
gdf_denue["category"].value_counts()

category
restaurants     8192
supermarkets    4647
hospitals        670
schools          499
metro            217
Name: count, dtype: int64

## Terrains

In [17]:
# read parquet
df_terrain_prices = pd.read_parquet("../../data/misc/terrain_prices.parquet")

# wrangle
df_terrain_prices = (
    df_terrain_prices
    .assign(
        mean_log_valor_fisico_terreno_m2_lower=lambda x: x["mean_log_valor_fisico_terreno_m2"] - z975 * x["std_log_valor_fisico_terreno_m2"],
        mean_log_valor_fisico_terreno_m2_upper=lambda x: x["mean_log_valor_fisico_terreno_m2"] + z975 * x["std_log_valor_fisico_terreno_m2"]
    )
    .assign(
        # fill nan with mean
        mean_log_valor_fisico_terreno_m2_lower=lambda x: np.where(
            x["mean_log_valor_fisico_terreno_m2_lower"].isna(),
            x["mean_log_valor_fisico_terreno_m2"],
            x["mean_log_valor_fisico_terreno_m2_lower"]
        ),
        mean_log_valor_fisico_terreno_m2_upper=lambda x: np.where(
            x["mean_log_valor_fisico_terreno_m2_upper"].isna(),
            x["mean_log_valor_fisico_terreno_m2"],
            x["mean_log_valor_fisico_terreno_m2_upper"]
        )
    )
    .drop(columns=["std_log_valor_fisico_terreno_m2", "city_cluster"])
)

# see
print(df_terrain_prices.shape)
df_terrain_prices.head()

(31392, 4)


,hex_id,mean_log_valor_fisico_terreno_m2,mean_log_valor_fisico_terreno_m2_lower,mean_log_valor_fisico_terreno_m2_upper
0,884982cb01fffff,6.831288,6.169717,7.492860
1,884982cb05fffff,7.158186,6.292182,8.024190
2,884982cb09fffff,6.831290,6.169714,7.492865
3,884982cb0bfffff,6.831288,6.169717,7.492860
4,884982cb0dfffff,7.158480,6.292533,8.024426


---
# Mega Merge

## Merge

In [18]:
def latlon_to_h3(latitudes, longitudes, resolution):
    """
    Convert latitude and longitude arrays to H3 hexagon IDs at a given resolution.
    
    Parameters:
    - latitudes: numpy array of latitudes
    - longitudes: numpy array of longitudes
    - resolution: H3 resolution
    
    Returns:
    - numpy array of H3 hexagon IDs
    """
    # Ensure inputs are numpy arrays
    latitudes = np.asarray(latitudes)
    longitudes = np.asarray(longitudes)
    
    # Check that latitudes and longitudes are the same shape
    if latitudes.shape != longitudes.shape:
        raise ValueError("Latitudes and longitudes must be the same shape")
    
    # Vectorize the h3 function
    vectorized_h3 = np.vectorize(h3.geo_to_h3)
    
    # Apply the function to the latitude and longitude arrays
    hex_ids = vectorized_h3(latitudes, longitudes, resolution)
    
    return hex_ids

# Get hex id for each property
HEX_RESOLUTION = 8
gdf_properties['hex_id'] = latlon_to_h3(
    gdf_properties['latitud'].values,
    gdf_properties['longitud'].values,
    HEX_RESOLUTION
)

In [19]:
# define function to get poiunts at r distance
def count_denues(gdf_props, gdf_denues, r=1):
    # fit a kdtree
    kdtree = KDTree(
        data=gdf_denues[['longitude', 'latitude']],
    )

    # count neighbors at r-km
    return kdtree.query_ball_point(
        gdf_props[['longitude', 'latitude']],
        r=r * 1_000,
        workers=-1,
        return_length=True
    )

# set crs to 6372
gdf_properties = gdf_properties.to_crs('EPSG:6372')
gdf_denue = gdf_denue.to_crs('EPSG:6372')

# get longitude & latitude
gdf_properties['longitude'] = gdf_properties.centroid.x
gdf_properties['latitude'] = gdf_properties.centroid.y

gdf_denue['longitude'] = gdf_denue.centroid.x
gdf_denue['latitude'] = gdf_denue.centroid.y

# count denues
dict_activities_search = {
    'supermarkets': 1,  # 1 km
    'hospitals': 5,
    'metro': 1,
    'schools': 1,
    'restaurants': 1
}

# count
for category, radius in tqdm(dict_activities_search.items()):
    gdf_properties[f'count_{category}_at_{radius}km'] = count_denues(
        gdf_properties,
        gdf_denue.query("category == @category"),
        r=radius
        )

# drop columns
gdf_properties = gdf_properties.drop(columns=['longitude', 'latitude'])

# see new cols
gdf_properties.iloc[:, -5:].head()

100%|██████████| 5/5 [00:01<00:00,  4.24it/s]


,count_supermarkets_at_1km,count_hospitals_at_5km,count_metro_at_1km,count_schools_at_1km,count_restaurants_at_1km
0,2,12,0,0,7
1,0,4,0,0,0
2,1,3,0,1,0
3,2,7,1,1,0
4,2,2,2,0,0


In [20]:
# merge all
gdf_properties2 = (
    gdf_properties
    # merge with enigh
    .assign(
        ubica_geo=lambda x: x["id_entidad_f"] + x["id_municipio"],
        year=lambda x: x["fecha_avaluo"].dt.year // 2 * 2 , # every 2 years
    )
    .merge(
        df_enigh,
        on=["ubica_geo", "year"],
        how="inner"
    )
    .drop(columns=["ubica_geo", "year"])
    # merge with banxico
    .assign(
        # get start of month of fecha_avaluo
        date=lambda x: x["fecha_avaluo"].dt.to_period("M").dt.to_timestamp(),
    )
    .merge(
        df_banxico,
        on=["date"],
        how="inner"
    )
    .drop(columns=["date"])
    # competitors
    .merge(
        df_competitors_stats,
        on="property_id",
        how="left"
    )
    # terrains
    .merge(
        df_terrain_prices,
        on="hex_id",
        how="inner"
    )
)

# print shape
print(gdf_properties2.shape)
print(gdf_properties2["property_id"].duplicated().sum())

gdf_properties2.head()

(852913, 93)
0


,property_id,valor_fisico_construccion,fecha_avaluo,colonia,id_clase_inmueble,id_tipo_inmueble,conservacion,elevador,niveles,cve_ref_proximidad_urbana,cve_nivel_equipamiento_urbano,valor_mercado,valor_concluido_total,nivel,unidades_rentables,edad_anios,vida_util_remanente,cve_clasificacion_zona,cve_vigilancia,regimen_propiedad,tipo_vialidad,id_municipio,id_entidad_f,cp,recamaras,banos,medio_banos,estacionamiento,superficie_terreno,superficie_construida,superficie_accesoria,superficie_vendible,valor_fisico_terreno_m2,property_type,city_cluster,distance_to_ocean,ocean_label,longitud,latitud,geometry,hex_id,count_supermarkets_at_1km,count_hospitals_at_5km,count_metro_at_1km,count_schools_at_1km,count_restaurants_at_1km,tam_loc,est_socio,clase_hog,sexo_jefe,edad_jefe,educa_jefe,tot_integ,mayores,menores,p12_64,p65mas,percep_ing,ing_cor,estim_alqu,otros_ing,gasto_mon,vivienda,alquiler,pred_cons,cuidados,tipo_viv,antiguedad,cuart_dorm,tenencia,renta,estim_pago,pago_viv,pago_mesp,tipo_adqui,viv_usada,tipo_finan,escrituras,tot_resid,tot_hom,tot_muj,tot_hog,total_viviendas,lag_year_inpc,lag_3months_em_avg_pct_change_inpc,lag_year_general_minimum_wage,lag_3months_em_avg_general_wage,competitors_weighted_mean_log_price_per_sqm,competitors_weighted_mean_log_price_per_sqm_lower,competitors_weighted_mean_log_price_per_sqm_upper,mean_log_valor_fisico_terreno_m2,mean_log_valor_fisico_terreno_m2_lower,mean_log_valor_fisico_terreno_m2_upper
0,66cf10199ef943a5a5ff82129e53d5d7,1611371.52,2020-05-27,SD,4,2,4.0,1.0,4,1,1,4305275,4305000,3,1,25,45,4,NaN,PRIVADA INDIVIDUAL,NaN,003,09,4600,2,1,0,0,143,348,0,348,8918.0,house,0,inf,no_ocean,-99.158103,19.310875,POINT (2797822.489 815572.062),884995843dfffff,2,12,0,0,7,1.0,3.013230,2.0,1.0,58.136311,7.407384,3.354268,3.087852,0.266416,2.530407,0.557445,2.304929,75369.418453,19406.663415,63.30374,38291.006045,4776.178204,1440.647648,1041.558150,2150.355388,1.0,25.374837,2.436469,4.0,515.588974,6647.702081,205.642062,0.139590,1.0,0.147720,5.0,1.0,3.378298,1.550932,1.827366,1.000000,225095.0,103.233,0.004525,115.9667,0.141828,9.719780,9.527069,9.912490,9.673058,8.712578,10.633539
1,f84f9a0c784f491eab6bb100b513a95b,460588.28,2020-07-23,SD,3,2,4.0,2.0,3,2,1,1351682,1352000,3,1,25,35,3,NaN,PRIVADA INDIVIDUAL,NaN,058,15,57100,2,2,0,1,113,108,0,108,6151.0,house,0,inf,no_ocean,-99.065361,19.491722,POINT (2807131.800 835725.131),884995b937fffff,0,4,0,0,0,1.0,2.285681,2.0,1.0,54.057086,6.417843,3.432375,2.914384,0.517991,2.510109,0.404276,1.823614,40971.566919,7871.843365,0.00000,28258.184054,3447.830493,1470.603904,318.300184,1058.354233,1.0,22.396067,2.165129,4.0,515.829785,2680.040383,0.000000,0.193193,2.0,0.101267,5.0,1.0,3.471221,1.658298,1.812923,1.011099,309044.0,103.687,-0.000685,115.4589,0.155422,9.565254,9.401056,9.729452,9.002990,8.205054,9.800925
2,7ddd5a94feed4955a809f986d95722d8,1029107.14,2020-10-05,SD,4,2,4.0,2.0,1,2,1,2544900,2545000,0,1,1,69,4,NaN,PRIVADA INDIVIDUAL,NaN,121,15,54743,3,2,0,1,200,170,0,170,5620.0,house,0,inf,no_ocean,-99.213541,19.633393,POINT (2791321.912 851047.443),884995aa9bfffff,1,3,0,1,0,1.0,3.191189,2.0,1.0,52.028625,7.157941,3.573237,3.102532,0.470705,2.919292,0.183240,1.967574,97548.637020,10937.011548,0.00000,42888.834613,4297.183586,1753.428238,482.131393,1960.737035,1.0,14.432705,2.300128,4.0,598.363736,3720.235750,93.162927,0.217279,1.0,0.159697,5.0,1.0,3.634310,1.751870,1.882440,1.012215,187317.0,104.503,0.003305,114.5573,0.159062,9.660476,9.521404,9.799547,8.285661,7.541603,9.029719
3,d2e75fb3b385461995bf8d34b9d1fdfb,598308.77,2020-10-22,SD,4,2,4.0,2.0,2,2,1,2906000,2906000,3,1,35,35,4,NaN,PRIVADA INDIVIDUAL,NaN,003,09,4480,6,2,0,0,107,139,0,139,18149.0,house,0,inf,no_ocean,-99.114525,19.333061,POINT (2802339.433 818111.830),88499585e1fffff,2,7,1,1,0,1.0,3.013230,2.0,1.0,58.136311,7.407384,3.354268,3.087852,0.266416,2.530407,0.557445,2.304929,75369.418453,19406.663415,63.30374,38291.006045,4776.178204,1440.647648,1041.558150,2150.355388,1.0,25.374837,2.436469,4.0,515.

## Nan values

In [21]:
# nan values
gdf_properties2.isnull().mean()[gdf_properties2.isnull().sum() > 0]

elevador                                             0.033259
cve_vigilancia                                       0.371019
tipo_vialidad                                        0.371019
competitors_weighted_mean_log_price_per_sqm          0.143140
competitors_weighted_mean_log_price_per_sqm_lower    0.143140
competitors_weighted_mean_log_price_per_sqm_upper    0.143140
dtype: float64

---
# Save

In [22]:
# shape
gdf_properties2.shape

(852913, 93)

In [23]:
# see cols
gdf_properties2.columns

Index(['property_id', 'valor_fisico_construccion', 'fecha_avaluo', 'colonia',
       'id_clase_inmueble', 'id_tipo_inmueble', 'conservacion', 'elevador',
       'niveles', 'cve_ref_proximidad_urbana', 'cve_nivel_equipamiento_urbano',
       'valor_mercado', 'valor_concluido_total', 'nivel', 'unidades_rentables',
       'edad_anios', 'vida_util_remanente', 'cve_clasificacion_zona',
       'cve_vigilancia', 'regimen_propiedad', 'tipo_vialidad', 'id_municipio',
       'id_entidad_f', 'cp', 'recamaras', 'banos', 'medio_banos',
       'estacionamiento', 'superficie_terreno', 'superficie_construida',
       'superficie_accesoria', 'superficie_vendible',
       'valor_fisico_terreno_m2', 'property_type', 'city_cluster',
       'distance_to_ocean', 'ocean_label', 'longitud', 'latitud', 'geometry',
       'hex_id', 'count_supermarkets_at_1km', 'count_hospitals_at_5km',
       'count_metro_at_1km', 'count_schools_at_1km',
       'count_restaurants_at_1km', 'tam_loc', 'est_socio', 'clase_hog',
  

In [24]:
# create folder
dir_save = "../../data/clean"
os.makedirs(dir_save, exist_ok=True)

# save
(
    gdf_properties2
    .drop(columns=["geometry"])
    .to_parquet(f"{dir_save}/properties_shif.parquet")
)

---
# Sandbox

In [25]:
# describe unidades_rentables
gdf_properties['tipo_vialidad'].value_counts(dropna=False, normalize=True)

tipo_vialidad
2.0    0.403138
NaN    0.371019
3.0    0.171999
5.0    0.019526
4.0    0.015394
1.0    0.011086
6.0    0.006482
8.0    0.000927
7.0    0.000412
0.0    0.000016
Name: proportion, dtype: float64

In [26]:
gdf_properties.fecha_avaluo.min()

Timestamp('2019-01-01 00:00:00')